<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#NCSES-class---FedRePORTER-and-IPEDS-data" data-toc-modified-id="NCSES-class---FedRePORTER-and-IPEDS-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>NCSES class - FedRePORTER and IPEDS data</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Python-Setup" data-toc-modified-id="Python-Setup-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Python Setup</a></span></li></ul></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the data</a></span><ul class="toc-item"><li><span><a href="#Federal-RePORTER---Abstracts-(https://federalreporter.nih.gov/FileDownload)" data-toc-modified-id="Federal-RePORTER---Abstracts-(https://federalreporter.nih.gov/FileDownload)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Federal RePORTER - Abstracts (<a href="https://federalreporter.nih.gov/FileDownload" target="_blank">https://federalreporter.nih.gov/FileDownload</a>)</a></span></li><li><span><a href="#NMF-method---Non-negative-matrix-factorization" data-toc-modified-id="NMF-method---Non-negative-matrix-factorization-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>NMF method - Non-negative matrix factorization</a></span></li></ul></li></ul></div>

## NCSES class - FedRePORTER and IPEDS data

### Introduction

**Federal RePORTER** (https://federalreporter.nih.gov) - a collaborative effort led by STAR METRICS® to create a searchable database of scientific awards from agencies (across agencies or fiscal years, by the award's project leader, or by a text search of a project's title, terms, or abstracts).

### Python Setup

In [1]:
# Data manipulation
import pandas as pd

# Reading in files
import glob

# Text analysis (topic modeling)
import numpy as np
import sklearn
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
import string

## Load the data

### Federal RePORTER - Abstracts (https://federalreporter.nih.gov/FileDownload)

In [2]:
"""Get all files with project abstracts."""

abstracts_files = glob.glob('FedRePORTER_PRJABS_C_FY20*.csv')
print(abstracts_files)

['FedRePORTER_PRJABS_C_FY2009.csv', 'FedRePORTER_PRJABS_C_FY2008.csv', 'FedRePORTER_PRJABS_C_FY2018.csv', 'FedRePORTER_PRJABS_C_FY2017.csv', 'FedRePORTER_PRJABS_C_FY2003.csv', 'FedRePORTER_PRJABS_C_FY2002.csv', 'FedRePORTER_PRJABS_C_FY2016.csv', 'FedRePORTER_PRJABS_C_FY2000.csv', 'FedRePORTER_PRJABS_C_FY2014.csv', 'FedRePORTER_PRJABS_C_FY2015.csv', 'FedRePORTER_PRJABS_C_FY2001.csv', 'FedRePORTER_PRJABS_C_FY2005.csv', 'FedRePORTER_PRJABS_C_FY2011.csv', 'FedRePORTER_PRJABS_C_FY2010.csv', 'FedRePORTER_PRJABS_C_FY2004.csv', 'FedRePORTER_PRJABS_C_FY2012.csv', 'FedRePORTER_PRJABS_C_FY2006.csv', 'FedRePORTER_PRJABS_C_FY2007.csv', 'FedRePORTER_PRJABS_C_FY2013.csv']


In [3]:
"""Read them in, concatenate and convert to a dataframe."""

list_data = []
for filename in abstracts_files:
    data = pd.read_csv(filename)
    list_data.append(data)
    
abstracts = pd.concat(list_data)

In [4]:
"""Drop missing abstracts."""

abstracts = abstracts.dropna()

In [5]:
"""Get abstracts as a list to feed in to TfidfVectorizer in the next step."""

merged_abstracts_list = abstracts[' ABSTRACT'].values.tolist()

### NMF method - Non-negative matrix factorization

NMF is a model used for topic extraction - while the LDA model uses raw counts of unique words per document, NMF model uses a normalized representation of those raw counts (TF-IDF representation)

TF stands for term-frequency and TF-IDF is term-frequency times inverse document-frequency. In other words, we are not only looking for how often a word appears in a given document, but also whether this particular word is distinct across all the collections of documents (corpus). For example, intuitively we understand that words like "often" or "use" are more frequently encountered, but they are less informative (more semantically-vacuous) if we want to discern a particular topic of a document, as they might be frequently encounter across all text documents in a corpus. On the other hand, words which we will see less frequently across a collection of document might indicate that those words are specific to a particular document, and, therefore, constitute a basis for a topic. 

More here: 

- https://scikit-learn.org/stable/modules/decomposition.html#non-negative-matrix-factorization-nmf-or-nnmf
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer

In [6]:
"""Convert a collection of raw documents to a matrix of TF-IDF features."""

vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(merged_abstracts_list)

In [7]:
"""Get feature names."""

vectorizer_feature_names = vectorizer.get_feature_names()

In [ ]:
"""Run the model with 100 topics."""

nmf = NMF(n_components=100, verbose=2).fit(tfidf)

In [ ]:
"""Get topics to documents and word to topics matrices."""

nmf_W = nmf.transform(tfidf) # get topics to documents matrix
nmf_H = nmf.components_ # get word to topics matrix

In [10]:
"""View the list of topics (10 top words per topic)"""

for topic_idx, topic in enumerate(nmf_H):
    print("Topic %d:" % (topic_idx))
    print('----------------------------')
    print(" ".join([vectorizer_feature_names[i]
                for i in topic.argsort()[:-10 - 1:-1]]))
    print('----------------------------')

Topic 0:
----------------------------
genes mutations identify candidate gene identified zebrafish genetic drosophila defects
----------------------------
Topic 1:
----------------------------
training trainees faculty postdoctoral biology fellows doctoral predoctoral scientists trainee
----------------------------
Topic 2:
----------------------------
subproject institution nih center isfor andinvestigator crisp theresources entries necessarily
----------------------------
Topic 3:
----------------------------
health disparities public mental policy minority population relevance racial ethnic
----------------------------
Topic 4:
----------------------------
cancer nci cancers pancreatic ovarian prevention oncology colorectal members colon
----------------------------
Topic 5:
----------------------------
hiv aids infected infection antiretroviral prevention transmission art cfar cd4
----------------------------
Topic 6:
----------------------------
core investigators services project

students graduate undergraduate student faculty college school underrepresented summer biomedical
----------------------------
Topic 56:
----------------------------
clinical trials translational trial phase protocol studies conduct monitoring basic
----------------------------
Topic 57:
----------------------------
cocaine addiction dopamine abuse da self relapse seeking reward administration
----------------------------
Topic 58:
----------------------------
genetic variants genome variation sequencing genomic genetics disease association traits
----------------------------
Topic 59:
----------------------------
administrative committee scientific advisory meetings management external activities oversight core
----------------------------
Topic 60:
----------------------------
vascular endothelial blood hypertension flow cardiovascular inflammation angiogenesis atherosclerosis inflammatory
----------------------------
Topic 61:
----------------------------
network networks wireless c

In [13]:
"""View a top document related to a given topic"""

for topic_idx, topic in enumerate(nmf_H):
    print('--------------------')
    print("Topic %d:" % (topic_idx))
    print('--------------------')
    print(" ".join([vectorizer_feature_names[i]
                    for i in topic.argsort()[:-10 - 1:-1]]))
    top_doc_indices = np.argsort(nmf_W[:,topic_idx] )[::-1][0:1]
    for doc_index in top_doc_indices:
        print('--------------------')
        print(merged_abstracts_list[doc_index])

--------------------
Topic 0:
--------------------
genes mutations identify candidate gene identified zebrafish genetic drosophila defects
--------------------
﻿   DESCRIPTION (provided by applicant): Our goal is to identify genes important to schizophrenia. In the first cycle of our project, we demonstrated that damaging de novo mutations in persons with schizophrenia, from otherwise healthy families, disrupt genes that are co-expressed in the dorsolateral and ventrolateral prefrontal cortex during fetal development. Compared to their unaffected siblings, schizophrenia patients were significantly more likely to harbor such alleles. Proteins encoded by these genes functioned in neuronal migration, synaptic transmission, signaling, and transcriptional regulation. Integration of genetic data and expression data suggested possible schizophrenia-related processes and even potential targets for treatment. In the next cycle of our project, we ask whether these and other candidate genes are e

cancer nci cancers pancreatic ovarian prevention oncology colorectal members colon
--------------------
The SEER Program was initiated in 1972 in response to requirements of the National Cancer Program for assessing the magnitude of the cancer burden in the United States, and for identifying factors related to cancer risk and/or patient survival. The SEER Program has among its objectives: 1. 	To assemble and report, on a periodic basis, estimates of cancer incidence, especially among the following key cancer sites: breast cancer, lung cancer, colorectal cancer, prostate cancer, pancreatic cancer, and urinary bladder cancer.2. 	To monitor annual cancer incidence trends to identify unusual changes in specific forms of cancer occurring in population subgroups defined by geographic, demographic, and social characteristics. 3. 	To provide continuing information on changes over time in extent of disease at diagnosis, trends in therapy, and changes in cancer patient survival. 4. 	To identify 

cells cd4 cd8 antigen nk epithelial cell differentiation progenitor effector
--------------------
White blood cells called T lymphocytes play critical roles in immune defense against viruses, bacteria, fungi, protozoa, and cancer cells. In the unactivated state, these cells circulate in the blood and accumulate in lymphoid tissues such as lymph nodes and spleen. Upon encounter with foreign materials (antigens) on the membranes of specialized antigen presenting cells (dendritic cells), these resting T-cells become activated, undergo numerous cell divisions, and differentiate into effector cells. The effector cells leave the lymphoid tissues and blood, entering sites of infection to combat pathogens. They can also invade normal tissues where their activity can cause autoimmune pathology. After elimination of an infecting organism, most of the activated T-cells die, but some remain as memory cells, to provide a more rapid and vigorous response if the same pathogen is encountered in the fu

--------------------
DESCRIPTION (provided by applicant): The success of neuronal cell replacement therapy depends on the ability of transplanted cells to synaptically integrate with host tissue. Complete integration requires that neurons can both send and receive synaptic information, as well as to modify their synaptic strength in response to changes in the cellular behavior of synaptically connected neurons. Due to limited cell tracking and stimulation techniques, previous reports have shown only that transplanted neurons can receive information from host neurons via synaptic stimulation. Thus, no direct evidence exists for their ability to send information to host cells or undergo synaptic plasticity. To test these hypotheses, we propose to use the light-activated Channelrhodopsin-2 (ChR2) ion channel linked to the mCherry fluorophore in human embryonic stem cell (hESC)-derived neurons. Following transplantation of hESC-derived forebrain-patterned neurons to the mouse hippocampus, 

--------------------
ABSTRACT NOT PROVIDED
--------------------
Topic 15:
--------------------
alcohol drinking use alcoholism consumption abuse dependence heavy substance binge
--------------------
alcohol
--------------------
Topic 16:
--------------------
stem differentiation hematopoietic progenitor hsc adult niche self regeneration transplantation
--------------------
DESCRIPTION (provided by applicant): Continuous replacement and repair of adult epithelial tissues such as the skin, intestine, and lung depend on self-renewing stem cells which generate the specialized cells necessary for tissue maintenance. Recent work has shown that the local environment, or niche, in which stem cells reside is critical for their maintenance and function. Specifically, positioning of the stem cell within the niche exposes it to signals that promote its survival and maintenance and guide the production of specialized daughter cells that perform the normal tissue functions. Self-renewing cells that 

...

women pregnancy maternal reproductive hpv pregnant birth fetal men ovarian
--------------------
Since 1995, over 40 countries have adopted legislation that mandates women's participation in government, and these policies enjoy considerable support from international organizations, politicians, and activists.  Practitioners who seek to increase women's presence in office do so in large part because they believe it will provide normative benefits for women through improved policy representation.  Despite this widely held assumption, it is clear that in a number of cases increasing women's representation fails to transform the legislative agenda.  While in some countries women's increased participation alters policy representation-defined in this project as the adoption of legislation related to women's interests-in others the addition of female legislators has had only limited consequences.Why does increasing women's participation lead to such mixed results?  This project represents one 

program members programs funding programmatic year scholars departments peer reviewed
--------------------
The Program Leaders for the Markey Cancer Center (MCC) provide cohesive leadership for promoting the research activities within and among the program areas. They assure continued interchange, program planning and evaluation, and educational exchange within their program. The Program Leaders identify areas of need within their research program and work with leadership of the cancer center to establish priorities and pursue recruitment in their respective areas. They are involved in identifying the shared resource needs of their program members and encourage collaborations between programs by communicating with leaders of other programs on possible areas of interactions and collaborations. They inform the appropriate Associate Director and the MCC Director on program activities and serve on the MCC's Scientific Advisory Panel and the Program and Shared Resource Leaders Committee. Th

--------------------
The development of HIV vaccines and other prevention strategies relies on the use of nonhuman primates in preclinical studies to advance the development of effective AIDS vaccine candidates and to advance development of effective topical microbicides and other prevention modalities or immune-based therapies. The NIAID Simian Vaccine Evaluation Unit (SVEU) contracts shall provide nonhuman primate resources that primarily support preclinical evaluation of AIDS vaccines. The SVEUs conduct studies in support of vaccines being developed by a wide range of investigators. These studies complement NIAID-supported basic vaccine research and vaccine evaluation studies funded through R01 (investigator-initiated research) grants, HIVRAD (HIV Vaccine Research and Design Program) grants, IPCAVD (Integrated Preclinical-Clinical AIDS Vaccine Development) grants, and NHP Consortium awards. The SVEUs also perform studies of candidate vaccines offered by companies or other researcher

iron heme fe hepcidin deficiency anemia metabolism metal transport overload
--------------------
﻿   DESCRIPTION (provided by applicant): As a cofactor for thousands of enzymes, iron is an essential micronutrient. Yet, free iron is toxic because it catalyzes rapid formation of damaging reactive oxygen species. Therefore, homeostasis systems exert tight control on iron levels in all organisms and gene expression is adjusted in response to iron deprivation and iron abundance: expression of at least 100 genes is known to be iron-dependent in Escherichia coli. In humans, disruption of iron homeostasis contributes to severe diseases: iron accumulation in the brain is linked to neurodegenerative diseases, iron overload causes the liver disease hemochromatosis, and iron deficiency leads to anemia and impaired cognitive development. Furthermore, invading bacterial pathogens hijack iron out of human proteins to establish infections. These considerations underscore the essential role of iron hom

projects spore project pilot developmental support investigators cores funds biostatistics
--------------------
The objectives of the Developmental Research Projects Program are to provide a continuous flow of new ideas and projects to stimulate myeloma research in the context of the Myeloma SPORE. It encourages new research directions and methodologies and facilitates collaborations. By providing initial support to pilot projects, it will foster the development of new translational projects. It also allows the Myeloma SPORE to have participation and recruitment of new investigators not only from the DF/HCC but also from outside institutions. Our Developmental Research Program during the previous funding period was extremely successful, with three projects in this renewal SPORE application directly evolving from prior Developmental Projects. This Program will continue to rely on scientific and programmatic review by the Governance Committee, which will assure selection of the most prom

As a result of the first topic model with 100 topics, 2 topics related to opioids and drug abuse are found.

In [14]:
"""Get topic weights per document."""

topics_weights = []
for index,i in enumerate(nmf_W): # for every document
    topics_weights.append([index, i[25], i[57]]) # get topic weights for 2 opioid- and drug abuse-related topics

In [15]:
"""Get those abstracts which have at least some value (not zero) for either of 2 topics."""

topics_list_dataframe = pd.DataFrame(topics_weights)

abstracts = abstracts.reset_index()
topics_list_dataframe = topics_list_dataframe.rename(columns={0:'index'})
concat = pd.concat([abstracts,topics_list_dataframe],axis=1)

filtered = concat[(concat[1] != 0) | (concat[2] != 0)] 

In [16]:
"""Get a list of abstracts from the filtered dataframe above."""

filtered_abstracts = filtered[' ABSTRACT'].values.tolist()

In [17]:
len(filtered_abstracts)

430340

In [ ]:
"""Run another topic model with 100 topics on the filtered abstracts."""

"""Convert a collection of raw documents to a matrix of TF-IDF features."""

vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(filtered_abstracts)

"""Get feature names"""
vectorizer_feature_names = vectorizer.get_feature_names()

"""Run the model with 100 topics"""
nmf = NMF(n_components=100, verbose=2).fit(tfidf)

In [21]:
"""Get topics to documents and word to topics matrices."""

nmf_W = nmf.transform(tfidf) # get topics to documents matrix
nmf_H = nmf.components_ # get word to topics matrix

violation: 1.0
violation: 0.5476233303884369
violation: 0.059274689694468134
violation: 0.01423237577474501
violation: 0.003332604999603872
violation: 0.0008879660903467299
violation: 0.0002619486792995731
violation: 6.933238294311101e-05
Converged at iteration 8


In [22]:
"""View the list of topics (10 top words per topic)"""

for topic_idx, topic in enumerate(nmf_H):
    print("Topic %d:" % (topic_idx))
    print('----------------------------')
    print(" ".join([vectorizer_feature_names[i]
                for i in topic.argsort()[:-10 - 1:-1]]))
    print('----------------------------')

Topic 0:
----------------------------
compounds inhibitors agents chemical novel activity new synthesis effects drugs
----------------------------
Topic 1:
----------------------------
training trainees faculty postdoctoral fellows scientists doctoral predoctoral medicine biology
----------------------------
Topic 2:
----------------------------
treatment intervention trial randomized efficacy cbt therapy treatments study placebo
----------------------------
Topic 3:
----------------------------
subproject institution nih center isfor andinvestigator crisp theresources entries necessarily
----------------------------
Topic 4:
----------------------------
hiv infected aids infection antiretroviral art prevention transmission msm tat
----------------------------
Topic 5:
----------------------------
cancer cancers nci pancreatic colon prevention colorectal bladder survivors members
----------------------------
Topic 6:
----------------------------
core projects investigators project core

management water soil production crop practices crops agricultural pest plant
----------------------------
Topic 57:
----------------------------
et al 2007 2005 2008 2006 2004 2009 2003 2010
----------------------------
Topic 58:
----------------------------
liver hepatic fibrosis hcc hbv nafld hepatitis fatty nash hepatocytes
----------------------------
Topic 59:
----------------------------
aging age older adults related aged changes life decline lifespan
----------------------------
Topic 60:
----------------------------
kidney renal ckd chronic cric injury aki hypertension esrd fibrosis
----------------------------
Topic 61:
----------------------------
signaling activation pathway aim pathways mechanisms induced role kinase receptor
----------------------------
Topic 62:
----------------------------
cardiac heart failure myocardial hf ventricular hypertrophy remodeling mi hearts
----------------------------
Topic 63:
----------------------------
program members programs year fun

In [23]:
"""View a top document related to a given topic"""

for topic_idx, topic in enumerate(nmf_H):
    print('--------------------')
    print("Topic %d:" % (topic_idx))
    print('--------------------')
    print(" ".join([vectorizer_feature_names[i]
                    for i in topic.argsort()[:-10 - 1:-1]]))
    top_doc_indices = np.argsort(nmf_W[:,topic_idx] )[::-1][0:1]
    for doc_index in top_doc_indices:
        print('--------------------')
        print(filtered_abstracts[doc_index])

--------------------
Topic 0:
--------------------
compounds inhibitors agents chemical novel activity new synthesis effects drugs
--------------------
1. To evaluate promising lead compounds for treating drug dependence through a broad panel of receptor and enzyme targets (Profile) in order to identify potential side effect liabilities and/or characterize the selectivity of the compounds for the desired pharmacological target set (Task 1). 2. To perform binding affinity and/or functional activity assessments of active compounds (Task 2). 3. To evaluate test compounds in a set of toxicological and pharmacokinetic assays in order to judge their potential for development as future therapeutic agents. These compounds and assays will be evaluated under Task 3. 4. Large compound libraries will be screened against a single target. (Task 4).
--------------------
Topic 1:
--------------------
training trainees faculty postdoctoral fellows scientists doctoral predoctoral medicine biology
------

--------------------
The Data Management/Statistical Core is a key component of the Center. The Core will serve as a support mechanism to the Center's overall research program. The overall objective of the Core is to facilitate the ability of the Center's investigators to conduct research that is of the highest standards and to disseminate the research outcomes to the academic and service communities and industry. The Core will provide statistical and analytical support to the research programs at all of the sites. The Core will also provide technical support for the research projects. In addition, the Core will serve as the depository for the cross-site core battery of measures. The Core will build on the structures developed in CREATE I for data collection, storage, transfer, and management, and quality control for the core battery. The methods developed in CREATE I will be further enhanced during the proposed CREATE II project to ensure that they reflect recent strategies in data ma

cognitive schizophrenia deficits measures control psychosis pfc prefrontal symptoms functional
--------------------
﻿   DESCRIPTION (provided by applicant): Cognitive deficits are an intrinsic part of schizophrenia, occurring independently of positive symptoms, and often persisting even when psychotic symptoms of schizophrenia have been successfully treated. Cognitive functioning is moderately to severely impaired in patients with schizophrenia and is typically present even in the prodromal phase of the disorder, in young drug-naïve patients. The deficits are in the domain of executive function largely controlled by the prefrontal cortex (PFC). However, there is only a fragmentary understanding of biochemical dysfunctions in brain that leads to cognitive impairment in schizophrenia. Furthermore, even though atypical antipsychotic drugs can improve certain aspects of cognition, many patients do not achieve remission. Therefore, the development of new therapeutic drugs for cognitive impa

--------------------
DESCRIPTION (provided by applicant): Hunter College proposes to continue, expand and improve its 27year old (1981) MBRS supported RISE program which has produced approximately 66 PhDs (38 in progress) from students who are underrepresented in science. In this Revision, we seek a supplement to support our Graduate Student Training program. In the competitive renewal of this grant (submitted May 2007), only the Undergraduate Research Training Program was supported. At both the undergraduate and graduate level, students who are underrepresented in science receive an intensive research experience in a laboratory engaged in nationally funded, competitive, state of the art, biomedical science research. At the undergraduate level (10 supported students), it also involves intensive mentoring by a faculty member and other enrichment activities intended to enhance science performance and to inspire and motivate students to seek advanced degrees in science (PhD) and a researc

--------------------
DESCRIPTION (provided by applicant): Type 2 diabetes is characterized by insulin resistance and disordered beta-cell function, especially a defect in glucose-stimulated insulin-secretion. Insulin receptors and signaling proteins are found within the beta-cells themselves. Recent evidence in rodents suggests the insulin signaling pathway is functional in islets and is important for glucose sensing. The physiologic role of insulin signaling within beta-cells in humans remains unknown. Our preliminary studies demonstrate that in healthy humans beta-cells are insulin sensitive tissues and raising circulating insulin levels by exogenous insulin administration can enhance glucose induced insulin secretion. We hypothesize that dysfunctional insulin signaling, i.e. insulin resistance, at the level of the beta-cell may be one mechanism underlying blunted insulin secretion in persons with type 2 diabetes and insulin resistant syndromes, which would be manifest as reduced ins

...

--------------------
DESCRIPTION (provided by applicant): The overall goal of The Clinical Profile of Parkinson's Disease (PD) Pathology, is to characterize the clinical profile of PD pathology in older person's without a diagnosis of PD. Showing that PD pathology is associated with a distinct and progressive condition among persons without a clinical diagnosis of PD, would have a transformative effect on PD studies. Although, PD only affects up to 5% of persons by age 85, compelling preliminary data shows that indices of PD pathology including nigral degeneration and Lewy bodies are present in nearly 20% of older persons without PD and are associated with the severity of parkinsonism proximate to death. This suggests that PD pathology is common and causes clinical signs in persons who do not meet clinical criteria for PD. Like the recent reclassification of AD, PD may also have an asymptomatic PD pathology phase, followed by a stage in which PD pathology results in mild motor and non-

--------------------
Biologists have been successful in revealing many of the molecular components involved in cell biology. Attention is now moving from identification of components to understanding the emergent, or system, behavior of the connected components. It has been hoped that general principles would emerge via the study of mathematical models of specific systems, but to date this form of study has not generally proven effective at discovering the hidden principles of biology. How the complex networks found in biological systems produce their emergent properties and behaviors remains elusive. Theoretical mathematics offers a possible route forward, and one which could, in time, have a profound influence on biology. This research project aims to cut through the complexity of biological models to elucidate the general principles of cell biology. Additionally, the project aims to develop new computational methods that can address currently infeasible problems. There is high deman

--------------------
﻿   DESCRIPTION (provided by applicant): Inflammatory bowel diseases are chronic and progressive inflammatory disorders that affect more than 1.4 million Americans. There is currently no cure for IBD, and over two-thirds of IBD patients become refractory to treatment with traditional therapies at some point in their lives and require surgical bowel resection. IBD i driven by inappropriate immune responses to the intestinal microbiota, which consists of trillions of bacteria that constitutively colonize the human intestine. The human gut microbiota shows considerable interindividual complexity and diversity with each person displaying a unique consortium of hundreds of bacterial species. Studies in mouse models of IBD have shown that select members of the microbiota have dramatic effects on colitis: some species exacerbate colitis by driving chronic inflammatory responses, while others protect against colitis by inducingimmunoregulatory responses. However, identifyi

--------------------
Our work focuses on two major research themes. One theme attempts to elucidate normative psychological and neurobiological mechanisms of fear and anxiety in healthy individuals in order to identify which of these mechanisms are perturbed in anxiety disorders. Anxiety is an adaptive response to threat that enables organisms to efficiently confront challenges either via rapid flight or fight responses (fear) or via sustained increased vigilance and behavioral inhibition (anxiety). When anxiety is evoked by anticipation of an unpleasant stimulus such as a mild electric shock (induced-anxiety), our sensorimotor system becomes primed to detect and react quickly to any threat. Our past work shows that this effect is mediated by increased communication between an aversive amplification circuit that includes the dorsal-medial prefrontal cortex (dmPFC) and the amygdala.  Phasic increased dmPFC-amygdala communication is responsible for our ability to rapidly detect and react

--------------------
Hepatitis C virus is both difficult to treat and a significant cause of morbidity and mortality. It is not understood how hepatitis C establishes infection nor how some patients clear infection. To this end 83 humans exposed to hepatitis C were studied in protocol 00-DK-0221. The immune response (both antibody and T cell) was closely followed and is being characterized. The study is in collaboration with Dr. Rehermann. As an extension of this 28 patients with acute hepatitis C have been followed and treated as indicated. Twenty-five patients (16 women; mean age 43 years) had adequate follow up to be included in an analysis.  Symptoms in the acute phase were reported by 80% and jaundice by 40%. Two patients (8%) developed ascites and acute liver failure but both survived.  Genotype 1 was most frequent (72%) but many patients (20%) could not be genotyped.   Serum aminotransferase levels and HCV RNA levels fluctuated greatly; 18% of patients were intermittently negati

4 related topics are found in a second round of a topic model with 100 topics.

In [24]:
"""Get abstracts with 4 related topics."""

topics_weights = []
for index,i in enumerate(nmf_W): # for every document
    topics_weights.append([index, i[8], i[11], i[33], i[53]]) # get topic weights for 4 related topics

topics_weights_dataframe = pd.DataFrame(topics_weights)

"""Sum the weights for all 4 related topics per document."""

topics_weights_dataframe['sum'] = topics_weights_dataframe[1] + topics_weights_dataframe[2] + topics_weights_dataframe[3] + topics_weights_dataframe[4]

In [32]:
filtered.columns = ['index','PROJECT_ID','ABSTRACT','index_copy',1,2]

In [37]:
filtered_abstracts = pd.DataFrame(filtered_abstracts)

In [38]:
filtered_abstracts = filtered_abstracts.reset_index()

In [ ]:
filtered_abstracts = filtered_abstracts.reset_index()

In [53]:
"""Filter by those abstracts who have a summed value of more than 0.01."""

filtered_by_threshold = topics_weights_dataframe[topics_weights_dataframe['sum'] >= 0.01]

"""Merge with the original dataframe. Make sure that level_0 column is in both dataframes to merge on."""

filtered_by_threshold = filtered_by_threshold.rename(columns={0:'level_0'})
filtered_by_threshold_finalized = filtered_by_threshold.merge(filtered_abstracts,on='level_0')

In [54]:
filtered_by_threshold_finalized = filtered_by_threshold_finalized.rename(columns={0:'ABSTRACT'})

In [55]:
filtered_by_threshold_updated = filtered_by_threshold_finalized.merge(filtered, on='ABSTRACT')

In [63]:
filtered_by_threshold_updated = filtered_by_threshold_updated[['PROJECT_ID','ABSTRACT']]

filtered_by_threshold_updated = filtered_by_threshold_updated.drop_duplicates()

"""Export results to .CSV"""

filtered_by_threshold_updated.to_csv('Filtered_Results_Abstracts.csv')